In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import csv
import json

In [ ]:
clients = pd.read_excel('CASE_CLIENTS.xlsx')
contracts = pd.read_excel('CASE_CONTRACTS.xlsx')
losses = pd.read_excel('CASE_LOSSES.xlsx')
exp_group = pd.read_excel('experiment_group.xlsx')

In [ ]:
r = requests.get('https://cbr.ru/scripts/XML_daily.asp?date_req=03/02/2024')

In [ ]:
with open ('cbr.xml', 'w') as f:
    f.write(r.text)

In [ ]:
def export_xml_to_csv_func():
    parser = ET.XMLParser(encoding="UTF-8")
    tree = ET.parse('cbr.xml', parser=parser)
    root = tree.getroot()

    with open('cbr.csv', 'w') as csv_file:
        writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for Valute in root.findall('Valute'):
            NumCode = Valute.find('NumCode').text
            CharCode = Valute.find('CharCode').text
            Nominal = Valute.find('Nominal').text
            Name = Valute.find('Name').text
            Value = Valute.find('Value').text
            writer.writerow([root.attrib['Date']] + [Valute.attrib['ID']] + [NumCode] + [CharCode] + [Nominal] +
                            [Name] + [Value.replace(',', '.')])

In [ ]:
export_xml_to_csv_func()

In [ ]:
df = pd.read_csv('cbr.csv', header=None)
df.head()

,0,1,2,3,4,5,6
0,03.02.2024,R01010,36,AUD,1,Австралийский доллар,59.8101
1,03.02.2024,R01020A,944,AZN,1,Азербайджанский манат,53.3309
2,03.02.2024,R01035,826,GBP,1,Фунт стерлингов Соединенного королевства,114.9783
3,03.02.2024,R01060,51,AMD,100,Армянских драмов,22.4412
4,03.02.2024,R01090B,933,BYN,1,Белорусский рубль,28.2324


In [ ]:
rate_df = df.rename(columns={
    0: 'rate_dt',
    2: 'currency_id',
    3: 'currency_cd',
    4: 'nominal_qty',
    6: 'currency_rate'})[['rate_dt', 'currency_id', 'currency_cd', 'nominal_qty', 'currency_rate']]
rate_df = rate_df[rate_df['currency_cd'] == 'USD']
rate_df.head()

,rate_dt,currency_id,currency_cd,nominal_qty,currency_rate
13,03.02.2024,840,USD,1,90.6626


In [ ]:
merged_data = pd.merge(contracts, clients, on='client_id')
merged_data = pd.merge(merged_data, losses, on='client_id')
merged_data = pd.merge(merged_data, exp_group, on='client_id')
merged_data = pd.concat([merged_data, rate_df], ignore_index=True)
merged_data.head(47)

,contract_id,contract_num,product_name,client_id,contract_status,currency_name,duration,country,price,insurance_amount,...,loss_id,loss_name,loss_payout_amt,group,experiment_id,rate_dt,currency_id,currency_cd,nominal_qty,currency_rate
0,2.111956e+08,ТТЕ7227715*****,Страхование путешественников,1.000071e+10,Действует,Российский рубль,10.0,Турция,1644.0,3000000.0,...,1.0,Оказание медицинской помощи,100000.0,control,policy_pricing,NaN,NaN,NaN,NaN,NaN
1,2.113238e+08,БТАСЕ27153*****,Страхование путешественников,1.000147e+08,Действует,Российский рубль,10.0,Турция,1644.0,3000000.0,...,2.0,Оказание медицинской помощи,50000.0,control,policy_pricing,NaN,NaN,NaN,NaN,NaN
2,2.115958e+09,ТБАТМСЛЕ47*****,Страхование путешественников USD,1.000579e+09,Действует,Доллар США,14.0,США,77.0,100000.0,...,45.0,Оказание медицинской помощи,20000.0,test,policy_pricing,NaN,NaN,NaN,NaN,NaN
3,2.116118e+09,ДСЖ-5/1539*****,Страхование путешественников,1.000655e+10,Действует,Российский рубль,10.0,Турция,1644.0,3000000.0,...,3.0,Оказание медицинской помощи,30000.0,control,policy_pricing,NaN,NaN,NaN,NaN,NaN
4,2.116191e+09,ИТЕ7 67777*****,Страхование путешественников,1.000661e+10,Действует,Российский рубль,10.0,Турция,1644.0,3000000.0,...,4.0,Оказание медицинской помощи,70000.0,control,policy_pricing,NaN,NaN,NaN,NaN,NaN
5,2.121233e+08,ДСЖ-5/1539*****,Страхование путешественников,1.001092e+09,Действует,Российский рубль,14.0,Турция,2301.0,3000000.0,...,5.0,Оказание медицинской помощи,60000.0,control,policy_pricing,NaN,NaN,NaN,NaN,NaN
6,2.121754e+08,ТМГО5Е7 № *****,Страхование путешественников,1.001199e+09,Действует,Российский рубль,14.0,Турция,2301.0,3000000.0,...,6.0,Оказание медицинской помощи,50000.0,control,policy_pricing,NaN,NaN,NaN,NaN,NaN
7,2.123544e+08,ТТЕ7227715*****,Страхование путешественников,1.001932e+08,Действует,Российский рубль,15.0,Турция,2466.0,3000000.0,...,7.0,Оказание медицинской помощи,100000.0,control,policy_pricing,NaN,NaN,NaN,NaN,NaN
8,2.124545e+08,ТТЕ7227778*****,Страхование путешественников,1.003457e+10,Действует,Российский рубль,30.0,Турция,4932.0,3000000.0,...,8.0,Оказание медицинской помощи,50000.0,control,policy_pricing,NaN,NaN,NaN,NaN,NaN
9,2.126338e+08,ДСЖ-5/1539*****,Страхование путешественников,1.004167e+10,Действует,Российский рубль,365.0,Турция,60000.0,3000000.0,...,9.0,Оказание медицинской помощи,20000.0,control,policy_pricing,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_data.columns

Index(['contract_id', 'contract_num', 'product_name', 'client_id',
       'contract_status', 'currency_name', 'duration', 'country', 'price',
       'insurance_amount', 'last_name', 'first_name', 'middle_name', 'age',
       'sex', 'loss_id', 'loss_name', 'loss_payout_amt', 'group',
       'experiment_id', 'rate_dt', 'currency_id', 'currency_cd', 'nominal_qty',
       'currency_rate'],
      dtype='object')

In [ ]:
merged_data = merged_data.drop(['contract_id', 'contract_num', 'product_name', 'client_id','contract_status', 'currency_name', 'duration', 'country','loss_id', 'loss_name', 'group',
                      'experiment_id', 'rate_dt', 'currency_id', 'nominal_qty'], axis = 1)

In [ ]:
merged_data = merged_data[['last_name', 'first_name', 'middle_name', 'age', 'sex','price', 'insurance_amount', 'loss_payout_amt', 'currency_cd', 'currency_rate']]
merged_data.head()

,last_name,first_name,middle_name,age,sex,price,insurance_amount,loss_payout_amt,currency_cd,currency_rate
0,A*****,A*****,P*****,61.0,F,1644.0,3000000.0,100000.0,NaN,NaN
1,А*****,Р*****,Э*****,67.0,F,1644.0,3000000.0,50000.0,NaN,NaN
2,Г*****,В*****,Я*****,45.0,M,77.0,100000.0,20000.0,NaN,NaN
3,Г*****,П*****,NaN,21.0,F,1644.0,3000000.0,30000.0,NaN,NaN
4,Г*****,Ю*****,К*****,83.0,M,1644.0,3000000.0,70000.0,NaN,NaN


In [ ]:
merged_data['currency_rate'] = merged_data['currency_rate'].replace(np.nan, 90.6626)
merged_data['currency_cd'] = merged_data['currency_cd'].replace(np.nan, 'USD')
merged_data.head()

,last_name,first_name,middle_name,age,sex,price,insurance_amount,loss_payout_amt,currency_cd,currency_rate
0,A*****,A*****,P*****,61.0,F,1644.0,3000000.0,100000.0,USD,90.6626
1,А*****,Р*****,Э*****,67.0,F,1644.0,3000000.0,50000.0,USD,90.6626
2,Г*****,В*****,Я*****,45.0,M,77.0,100000.0,20000.0,USD,90.6626
3,Г*****,П*****,NaN,21.0,F,1644.0,3000000.0,30000.0,USD,90.6626
4,Г*****,Ю*****,К*****,83.0,M,1644.0,3000000.0,70000.0,USD,90.6626


In [ ]:
merged_data.loc[~merged_data.index.isin([2, 15]), 'price_usd'] = merged_data['price'] / merged_data['currency_rate']# Выбираем строки 2 и 15 и заменяем значение 'price_usd' только в оставшихся строках
merged_data.at[2, 'price_usd'] = 77
merged_data.at[15, 'price_usd'] = 33

merged_data.head()

,last_name,first_name,middle_name,age,sex,price,insurance_amount,loss_payout_amt,currency_cd,currency_rate,price_usd
0,A*****,A*****,P*****,61.0,F,1644.0,3000000.0,100000.0,USD,90.6626,18.133166
1,А*****,Р*****,Э*****,67.0,F,1644.0,3000000.0,50000.0,USD,90.6626,18.133166
2,Г*****,В*****,Я*****,45.0,M,77.0,100000.0,20000.0,USD,90.6626,77.000000
3,Г*****,П*****,NaN,21.0,F,1644.0,3000000.0,30000.0,USD,90.6626,18.133166
4,Г*****,Ю*****,К*****,83.0,M,1644.0,3000000.0,70000.0,USD,90.6626,18.133166


In [ ]:
merged_data.loc[~merged_data.index.isin([2, 15]), 'insurance_amount_usd'] = merged_data['insurance_amount'] / merged_data['currency_rate']# Выбираем строки 2 и 15 и заменяем значение 'price_usd' только в оставшихся строках
merged_data.at[2, 'insurance_amount_usd'] = 100000
merged_data.at[15, 'insurance_amount_usd'] = 50000
merged_data.head()

,last_name,first_name,middle_name,age,sex,price,insurance_amount,loss_payout_amt,currency_cd,currency_rate,price_usd,insurance_amount_usd
0,A*****,A*****,P*****,61.0,F,1644.0,3000000.0,100000.0,USD,90.6626,18.133166,33089.719465
1,А*****,Р*****,Э*****,67.0,F,1644.0,3000000.0,50000.0,USD,90.6626,18.133166,33089.719465
2,Г*****,В*****,Я*****,45.0,M,77.0,100000.0,20000.0,USD,90.6626,77.000000,100000.000000
3,Г*****,П*****,NaN,21.0,F,1644.0,3000000.0,30000.0,USD,90.6626,18.133166,33089.719465
4,Г*****,Ю*****,К*****,83.0,M,1644.0,3000000.0,70000.0,USD,90.6626,18.133166,33089.719465


In [ ]:
merged_data.loc[~merged_data.index.isin([2, 15]), 'loss_payout_amt_usd'] = merged_data['loss_payout_amt'] / merged_data['currency_rate']# Выбираем строки 2 и 15 и заменяем значение 'price_usd' только в оставшихся строках
merged_data.at[2, 'loss_payout_amt_usd'] = 20000
merged_data.at[15, 'loss_payout_amt_usd'] = 30000
merged_data.head()

,last_name,first_name,middle_name,age,sex,price,insurance_amount,loss_payout_amt,currency_cd,currency_rate,price_usd,insurance_amount_usd,loss_payout_amt_usd
0,A*****,A*****,P*****,61.0,F,1644.0,3000000.0,100000.0,USD,90.6626,18.133166,33089.719465,1102.990649
1,А*****,Р*****,Э*****,67.0,F,1644.0,3000000.0,50000.0,USD,90.6626,18.133166,33089.719465,551.495324
2,Г*****,В*****,Я*****,45.0,M,77.0,100000.0,20000.0,USD,90.6626,77.000000,100000.000000,20000.000000
3,Г*****,П*****,NaN,21.0,F,1644.0,3000000.0,30000.0,USD,90.6626,18.133166,33089.719465,330.897195
4,Г*****,Ю*****,К*****,83.0,M,1644.0,3000000.0,70000.0,USD,90.6626,18.133166,33089.719465,772.093454


In [165]:
#Создание новой колонки для суммы по убытку(Loss_payout_sum)
loss_payout_usd_sum = merged_data['loss_payout_amt_usd'].sum()
print("Сумма по убытку: ", loss_payout_usd_sum)

Сумма по убытку:  159968.1676898743


In [ ]:
merged_data.to_excel('merged_file.xlsx', index=False)